In [3]:
import os, sys, time, subprocess
from collections import defaultdict
import datetime as dt
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
mypath = os.path.dirname(os.path.realpath('.')) #'__file__'
sys.path.insert(0,mypath)
from whatsapp import *

[      ____  __  __   _          
      / __ \|  \/  | | |         
     | |  | | \  / | | |         
     | |  | | |\/| | | |         
     | |__| | |  | | | |         
      \____/\_|  |_| |_|        ]

---------------------------------


In [4]:

def sidepane_scroll_by_id(driver, initial = 1, scrolls=25):
    for i in range(0, scrolls):
        driver.execute_script("document.getElementById('pane-side').scrollTop={}".format(initial))
        initial += 300
    return initial

def scroll_by_xpath(driver, xpt="(//div[@id='main']//div[@class='ItfyB _3nbHh'])[2]"):
    wait = WebDriverWait(driver, 30)
    driver.execute_script("arguments[0].scrollIntoView(true);", wait.until(EC.presence_of_element_located((By.XPATH, xpt))))
    return 1
    
def reload(driver, wait_sec=60):
    wait = WebDriverWait(driver, wait_sec)
    driver.refresh()
    search_bar_xpt = "/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/div[2]/div/div[2]"
    element = wait.until(EC.presence_of_element_located((By.XPATH, search_bar_xpt)))
    print('reload success, whatsapp ready')

def xp(driver, xpt): #presence
    try:
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
        return element
    except: return False

def xfill(driver, xpt, txt=None, search_only =True):
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.presence_of_element_located((By.XPATH, xpt))) if type(xpt) is str else xpt
    if element != False:
        element.clear()
        ac = ActionChains(driver)
        ac.move_to_element(element).click().perform()
        if txt is not None and chr(10) in txt and search_only==False:
            xx = txt.split(chr(10))
            for line in xx:
                ac.send_keys(line).perform()
                ac.key_down(Keys.SHIFT).key_down(Keys.ENTER).key_up(Keys.SHIFT).key_up(Keys.ENTER).perform()
            ac.send_keys(Keys.RETURN).perform()
        elif txt is not None and search_only==False:
            ac.send_keys(name).send_keys(Keys.RETURN).perform()
        elif txt is not None and search_only==True:
            ac.send_keys(name).perform()
            
def select_chat(driver, name, search_bar_xpt = "/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/div[2]/div/div[2]"):
    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.presence_of_element_located((By.XPATH, search_bar_xpt)))
    element.clear()
    ac = ActionChains(driver)
    ac.move_to_element(element).click().perform()
    ac.send_keys(name).send_keys(Keys.RETURN).perform()
    

def ca(driver, xpath:dict):
    wait = WebDriverWait(driver, 3)
    ac = ActionChains(driver)
    def xpg(xpt):
        try: return wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
        except: return None
    for xpt, cmd in xpath.items():
        el = xpg(xpt)
        if el is not None:
            a = cmd[0]
            if a =='hover-click'or a == 1:
                ac.move_to_element(el).click(xpg(cmd[1])).perform()
            elif a =='click' or a == 2 and (ac is None or ac ==''):
                ac.move_to_element(el).click()
            elif a =='paste enter' or a==3 and ac is not None :
                ac.send_keys(a[1]).send_keys(Keys.RETURN).perform()
            elif a =='paste-only' or a == 4 and ac is not None:
                ac.send_keys(a[1]).perform()
            else: pass

        
def xc(driver, xpt, click=True): #clickable
    try:
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.element_to_be_clickable((By.XPATH, xpt)))
        if click: element.click()
        else: return element
    except: return False

def xp(driver, xpt): #presence
    try:
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
        print('element found and returned')
        return element
    except: return False

def xt(driver, xpt):
    try:
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.presence_of_element_located((By.XPATH, xpt))).text
        print(xpt, " Text: ", element)
        return element
    except: return False
    

panexpt = {'chat' : ["//div[1]/div[1]/span"],
        'lastmsg' : ["//div[1]/span/span[1]"],
        'msg_sender': ["//div[2]/div[1]/span/span[3]"],
        'dttm' : ["//div[@class='Dvjym']"]}



In [5]:
def dict_update(cond, ky, vl):
    '''update dictionary values if key already exist otherwise add new into dictionary, single procedure once'''
    if type(vl) is list:
        cond[ky] = vl if not list(cond) else cond.get(ky, []) + vl
    else:
        cond[ky] = [vl] if not list(cond) else cond.get(ky, []) + [vl]
    return cond


def update_key_value(a, b):
    '''updating key & values (values will be appened for existing key)'''
    c = {}
    tls = lambda ls : ls if type(ls) is list else [ls]
    for k, v in a.items():
        c[k] = tls(a[k]) + tls(b[k]) if k in list(b) else tls(a[k])
    else: return {**b, **c}

In [6]:


class Wakit(whapp):
    
    def __init__(self):
        chrome_options = Options()
        chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        self.wait = WebDriverWait(self.driver, 5)
        self.cdata = defaultdict()
        self.parsed_counter = 0
        print(self.driver.title)
        
    def isPresent(self, xpt):
        try: return self.wait.until(EC.presence_of_element_located((By.XPATH, xpt)))
        except: return False
    
    def FetchText(self, xpt):
        try: return self.wait.until(EC.presence_of_element_located((By.XPATH, xpt))).text
        except: return None
        
    def writer(self, data, method='a+'):
        if data is not None:
            data = data.replace(char(10),char(164))
            file = 'XPT_' + str(dt.datetime.now()) + '.txt'
            fp = open('output\\' + file, method)
            fp.write(data + "##" + chr(10))
            fp.close()
        
    def iterchat(self, basexpt, xptfn, start_index= 2, required=20):
        miss = 0
        for n in range(start_index, required):
            xpd = xptfn(n, basexpt)            
            t = self.FetchText(xpd['base'])
            
            if t == False:
                if miss>3: return 1
                else: miss = miss + 1
            elif t is not None and t not in self.btext:
                self.btext.append(t)
                for k, v in xpd.items():
                    for n in v:
                        opt = [str(n) + "::" + str(self.FetchText(n)) for n in v]
                        self.cdata = update_key_value(self.cdata,k,opt)
                        self.writer(k + ':::' + opt)
                        self.parsed_counter = self.parsed_counter + 1
                        miss = 0
            else:
                pass
        
    def chatparse(self, basexpt, xptfn, start_index= 2, required=20):
        self.parsed_counter = 0
        self.btext = []
        while self.parsed_counter < required:
            s = self.iterchat(basexpt, xptfn, start_index, required)
            scroll_by_xpath(basexpt + '[2]')
        print('parsed: ', self.parsed_counter)
    
    @property
    def get_driver(self):
        return self.driver
            
        

In [7]:
x = Wakit()
d = x.get_driver
reload(d)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████▋| 6.55M/6.58M [00:05<00:00, 651kB/s]

(26) WhatsApp
element found


[WDM] - Downloading: 100%|█████████████████████████████████████████████████████████| 6.58M/6.58M [00:20<00:00, 651kB/s]

In [ ]:
df = pd.DataFrame({'Squares':[4, 9, 16, 25 ],'Cubes':[8, 27, 64, 125]})
for i in range(6,10):
    df=df.append({'Squares': i**2, 'Cubes': i**3}, ignore_index=True)

In [25]:
i = sidepane_scroll_by_id(d, initial = 1, scrolls=25)

In [26]:
i = sidepane_scroll_by_id(d, initial = i, scrolls=25)

In [24]:
select_chat(d, 'emergency')
scroll_by_xpath(d, xpt="(//div[@id='main']//div[@class='ItfyB _3nbHh'])[1]")

1

In [21]:
scroll_by_xpath(d, xpt="(//div[@id='main']//div[@class='ItfyB _3nbHh'])[2]")

1

In [22]:
scroll_by_xpath(d, xpt="(//div[@id='main']//div[@class='ItfyB _3nbHh'])[2]")

1

In [ ]:
def bodymsg(nodeno=2, bodybase = "(//div[@class='ItfyB _3nbHh'])"):
    base = bodybase + "[" + str(nodeno) + ']'
    bodyxpt = {'sender':[base + '/div/div[1]'],
               'sender_text':[ base + '/div/div[2]'],
               'send_time':[base + '/div/div[3]'],
               'Q_Clickable':[base + '/div[1]/div/div/div/div'],
               'Q_Sender':[base + '//div[2]/div[1]/div/div/div/div/div[1]/span'],
               'Q_Text':[base + '//div[2]/div[1]/div/div/div/div/div[2]/span']}
    bodyxpt['base'] = [base]
    return bodyxpt

def sidepane(nodeno=2, panebase = "(//div[@id='pane-side']//div[@class='_3OvU8'])"):
    base = panebase + "[" + str(nodeno) + "]"
    panexpt = {'chat' : [base + "//div[1]/div[1]/span"],
            'lastmsg' : [base + "//div[1]/span/span[1]"],
            'msg_sender': [base + "//div[2]/div[1]/span/span[3]"],
            'dttm' : [base + "//div[@class='Dvjym']"]}
    panexpt['base'] = [base]
    return panexpt

def iterdict(driver, xpd:dict, sec=3):
    rd = defaultdict()
    file = 'XPT_' + dt.datetime.now().strftime("%H%M %Y%m%d") + '.txt'
    fp = open(os.getcwd() + '\\output\\' + file, 'a')
    for k, v in xpd.items():
        rls = []
        for n in v:
            wait = WebDriverWait(driver, sec)
            try: rs = wait.until(EC.presence_of_element_located((By.XPATH, n))).text
            except: rs = False
            print(str(k) + chr(10) + str(n) + '::' + str(rs) + chr(10) + chr(10))
            fp.write(str(k) + chr(10) + str(n) + '::' + str(rs) + chr(10) + chr(10))
            rls.append(rs)
        rd = dict_update(rd, k, rls)
    else: 
        fp.close()
        return rd
            


In [ ]:

#xfill(d,"/html/body/div[1]/div/div/div[3]/div/div[1]/div/div/div[2]/div/div[2]","SM & CN_Monitoring")
#x.chatparse("(//div[@id='pane-side']//div[@class='_3OvU8'])",sidepane_xpt)


def iternode(driver, base_xpt, start_node, end_node, parsefn, scrollfn=None, ini=1, sc=25):
    wait = WebDriverWait(driver, 5)
    for n in range(start_node, end_node):
        x = base_xpt + '[' + str(n) + ']'
        try:
            el = wait.until(EC.presence_of_element_located((By.XPATH, x)))
        except:
            ini = scrollfn(driver, initial=ini, scrolls=sc)
            iternode(driver, base_xpt, start_node, end_node, scrollfn, ini, sc)
    else:
        print('data available')
        return 1

In [ ]:
msg_base = "(//div[@class='ItfyB _3nbHh'])"

In [ ]:
a = {'chat_name' : ["//span[@class='_3q9s6']", "//div[@class='zoWT4']}